In [1]:
pip install transformers torch scikit-learn huggingface_hub pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Import necessary libraries
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip install pandas

     ---------------------------------------- 11.6/11.6 MB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 15.9/15.9 MB 6.9 MB/s eta 0:00:00
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
     -------------------------------------- 507.9/507.9 KB 6.4 MB/s eta 0:00:00
     -------------------------------------- 346.8/346.8 KB 5.3 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [4]:
# Load the dataset
dataset_path = 'D:\Sentiment_analysis\Sentiment_analysis\009.csv'
df = pd.read_csv(dataset_path)

ValueError: embedded null character

In [9]:
# Rename columns to standardize with the dataset
df.columns = ['text', 'label']

NameError: name 'df' is not defined

In [10]:
# Encode labels
label_to_id = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_to_id)

NameError: name 'df' is not defined

In [11]:

# Split dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42)

NameError: name 'df' is not defined

In [7]:
# Define Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [8]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-mini")
model = BertForSequenceClassification.from_pretrained("prajjwal1/bert-mini", num_labels=len(label_to_id))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Make model parameters contiguous to avoid the non-contiguous tensor error
for param in model.parameters():
    param.data = param.data.contiguous()


In [10]:
# Create dataset objects
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

In [ ]:
# Define training arguments with a custom run_name to avoid the W&B warning
training_args = TrainingArguments(
    output_dir='./results',
    run_name="bert-mini-sentiment-analysis",  # Custom run name
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Train the model
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.976500,1.071098
2,0.913500,0.994060
3,0.831100,0.979748


TrainOutput(global_step=19620, training_loss=1.0908889168020903, metrics={'train_runtime': 670.0396, 'train_samples_per_second': 468.442, 'train_steps_per_second': 29.282, 'total_flos': 778293891360000.0, 'train_loss': 1.0908889168020903, 'epoch': 3.0})

In [13]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 0.9797477722167969,
 'eval_runtime': 24.2368,
 'eval_samples_per_second': 1079.228,
 'eval_steps_per_second': 67.459,
 'epoch': 3.0}

In [14]:
model.save_pretrained("./sentiment-classification-bert-mini")
tokenizer.save_pretrained("./sentiment-classification-bert-mini")


('./sentiment-classification-bert-mini/tokenizer_config.json',
 './sentiment-classification-bert-mini/special_tokens_map.json',
 './sentiment-classification-bert-mini/vocab.txt',
 './sentiment-classification-bert-mini/added_tokens.json')

In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("./sentiment-classification-bert-mini")  # Or use "your-username/sentiment-classification-bert-mini" if it's on Hugging Face Hub
tokenizer = BertTokenizer.from_pretrained("./sentiment-classification-bert-mini")

# Define a label map to decode the output (adjust according to your dataset's label encoding)
label_map = {0: "sadness", 1: "anger", 2: "love"}  # Replace with your actual labels

# Prediction function
def predict_label(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)

    # Get the predicted label
    logits = outputs.logits
    predicted_label = torch.argmax(logits, dim=1).item()
    return label_map[predicted_label]

# Test the function with example input
test_sentence = "I feel humiliated and sad."
print(f"Sentence: {test_sentence}")
print(f"Predicted label: {predict_label(test_sentence)}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence: I feel humiliated and sad.
Predicted label: sadness


In [16]:
from huggingface_hub import notebook_login

notebook_login()


In [17]:
# Push the model
model.push_to_hub("Varnikasiva/sentiment-classification-bert-mini")

# Push the tokenizer
tokenizer.push_to_hub("Varnikasiva/sentiment-classification-bert-mini")


model.safetensors:   0%|          | 0.00/44.7M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Varnikasiva/sentiment-classification-bert-mini/commit/aa0b765af94687fa80b5180dac0f83a716e2608e', commit_message='Upload tokenizer', commit_description='', oid='aa0b765af94687fa80b5180dac0f83a716e2608e', pr_url=None, pr_revision=None, pr_num=None)